In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote


In [3]:
# Input Search Query
SEARCH_QUERY = 'tv'

In [4]:
PROXY_CRAWL_TOKEN = '8wJqpL2FErGiPGX7QWP6Aw'

In [5]:
def get_proxy(url):
    quoted_url = quote(url)
    res = requests.get(f'https://api.proxycrawl.com/?token={PROXY_CRAWL_TOKEN}&url={quoted_url}')
    res.raise_for_status() # Raise error if it fails
    return res.content

In [6]:

#### A function to get the content of the page of required query
def search_in_amazon(search_query):
    url = f"https://www.amazon.com/s?k={search_query}"
    return get_proxy(url)

#### A function to get the contents of individual product pages using 'data-asin' number (unique identification number)
def search_asin(asin):
    url = f"https://www.amazon.com/dp/{asin}"
    return get_proxy(url)

#### A function to pass on the link of 'see all reviews' and extract the content
def search_reviews(review_link):
    url = f"https://www.amazon.com{review_link}"
    return get_proxy(url)


### Product Name extraction

In [7]:
print("Start to extract product names")


product_names = []
data_asin = []
LAST_PAGE = 21
for i in range(1,LAST_PAGE):
    print(f"Iteration {i}/{LAST_PAGE}")
    html = search_in_amazon(SEARCH_QUERY+'&page='+str(i))
    soup = BeautifulSoup(html, 'html5lib')
    items = soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'})
    if len(items) == 0:
        print(f"Warning: No product name found in this page")
    for i in items:
        product_names.append(i.text) # adding the product names to the list

    for i in soup.findAll("div", {"class":"s-result-item"}):
        if i['data-asin']:
            data_asin.append(i['data-asin'])

print(f"Finished: {len(product_names)} product names found")


Start to extract product names
Iteration 1/21
Iteration 2/21
Iteration 3/21
Iteration 4/21
Iteration 5/21
Iteration 6/21
Iteration 7/21
Iteration 8/21
Iteration 9/21
Iteration 10/21
Iteration 11/21
Iteration 12/21
Iteration 13/21
Iteration 14/21
Iteration 15/21
Iteration 16/21
Iteration 17/21
Iteration 18/21
Iteration 19/21
Iteration 20/21
Finished: 306 product names found


In [8]:
len(product_names)

306

When scrawling the all pages of product list in specific search query, I could discover that there are same products in the list. <br>
Therefore, I needed to remove the same product in the list of ASIN.

In [9]:
print("Start to search asin")

Start to search asin


In [10]:


asin_list = list(dict.fromkeys(data_asin))
data_asin = asin_list

link=[]
for i in range(len(data_asin)):
    print(f"Searching asin {i}/{len(data_asin)}")
    html = search_asin(data_asin[i])
    soup = BeautifulSoup(html, 'html5lib')
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

Searching asin 0/247
Searching asin 1/247
Searching asin 2/247
Searching asin 3/247
Searching asin 4/247
Searching asin 5/247
Searching asin 6/247
Searching asin 7/247
Searching asin 8/247
Searching asin 9/247
Searching asin 10/247
Searching asin 11/247
Searching asin 12/247
Searching asin 13/247
Searching asin 14/247
Searching asin 15/247
Searching asin 16/247
Searching asin 17/247
Searching asin 18/247
Searching asin 19/247
Searching asin 20/247
Searching asin 21/247
Searching asin 22/247
Searching asin 23/247
Searching asin 24/247
Searching asin 25/247
Searching asin 26/247
Searching asin 27/247
Searching asin 28/247
Searching asin 29/247
Searching asin 30/247
Searching asin 31/247
Searching asin 32/247
Searching asin 33/247
Searching asin 34/247
Searching asin 35/247
Searching asin 36/247
Searching asin 37/247
Searching asin 38/247
Searching asin 39/247
Searching asin 40/247
Searching asin 41/247
Searching asin 42/247
Searching asin 43/247
Searching asin 44/247
Searching asin 45/24

In [11]:
print(f"Finished: {len(link)} links found")
# The number of link and the number of ASIN can be different, because there are many products which have no review.

Finished: 213 links found


In [34]:

print("Start to search reviews")
# urls=[]
# titles = []
search_query_list = []
reviews=[]
# ratings=[]
# dates=[]



Start to search reviews


In [35]:


for j in range(len(link)):
    print(f"Searching reviews: {j}/{len(link)}")
    for k in range(1, 1500):
        print(f"Finished: {len(reviews)} reviews found")
        html = search_reviews(link[j]+'&pageNumber='+str(k))
        soup = BeautifulSoup(html, 'html5lib')
        if soup.find(
            'div',
            {"class" : "a-section a-spacing-top-large a-text-center no-reviews-section"}
        ):
            print('No review, Pass') 
            break
        else:
            items = soup.select('span[data-hook="review-body"] > span')
            print(f"{len(items)} reviews found")
            for i in items:
                if i.text== '':
                    "there is one blank span element"
                    continue
                else:
                    reviews.append(i.text)
                    search_query_list.append(SEARCH_QUERY)

print(f"Finished: {len(reviews)} reviews found")



Searching reviews: 0/213
Finished: 0 reviews found
10 reviews found
Finished: 10 reviews found
10 reviews found
Finished: 20 reviews found
10 reviews found
Finished: 30 reviews found
10 reviews found
Finished: 40 reviews found
11 reviews found
Finished: 50 reviews found
10 reviews found
Finished: 60 reviews found
10 reviews found
Finished: 70 reviews found
10 reviews found
Finished: 80 reviews found
11 reviews found
Finished: 90 reviews found
10 reviews found
Finished: 100 reviews found
10 reviews found
Finished: 110 reviews found
10 reviews found
Finished: 120 reviews found
10 reviews found
Finished: 130 reviews found
10 reviews found
Finished: 140 reviews found
10 reviews found
Finished: 150 reviews found
10 reviews found
Finished: 160 reviews found
10 reviews found
Finished: 170 reviews found
10 reviews found
Finished: 180 reviews found
10 reviews found
Finished: 190 reviews found
10 reviews found
Finished: 200 reviews found
10 reviews found
Finished: 210 reviews found
10 reviews fo

10 reviews found
Finished: 1810 reviews found
10 reviews found
Finished: 1820 reviews found
10 reviews found
Finished: 1830 reviews found
10 reviews found
Finished: 1840 reviews found
10 reviews found
Finished: 1850 reviews found
10 reviews found
Finished: 1860 reviews found
10 reviews found
Finished: 1870 reviews found
10 reviews found
Finished: 1880 reviews found
10 reviews found
Finished: 1890 reviews found
10 reviews found
Finished: 1900 reviews found
10 reviews found
Finished: 1910 reviews found
10 reviews found
Finished: 1920 reviews found
10 reviews found
Finished: 1930 reviews found
10 reviews found
Finished: 1940 reviews found
11 reviews found
Finished: 1950 reviews found
10 reviews found
Finished: 1960 reviews found
11 reviews found
Finished: 1970 reviews found
10 reviews found
Finished: 1980 reviews found
10 reviews found
Finished: 1990 reviews found
10 reviews found
Finished: 2000 reviews found
10 reviews found
Finished: 2010 reviews found
10 reviews found
Finished: 2020 re

10 reviews found
Finished: 3600 reviews found
10 reviews found
Finished: 3610 reviews found
11 reviews found
Finished: 3620 reviews found
11 reviews found
Finished: 3630 reviews found
10 reviews found
Finished: 3640 reviews found
10 reviews found
Finished: 3650 reviews found
11 reviews found
Finished: 3660 reviews found
10 reviews found
Finished: 3670 reviews found
10 reviews found
Finished: 3680 reviews found
12 reviews found
Finished: 3690 reviews found
11 reviews found
Finished: 3700 reviews found
10 reviews found
Finished: 3710 reviews found
10 reviews found
Finished: 3720 reviews found
10 reviews found
Finished: 3730 reviews found
11 reviews found
Finished: 3740 reviews found
10 reviews found
Finished: 3750 reviews found
10 reviews found
Finished: 3760 reviews found
11 reviews found
Finished: 3770 reviews found
10 reviews found
Finished: 3780 reviews found
10 reviews found
Finished: 3790 reviews found
10 reviews found
Finished: 3800 reviews found
10 reviews found
Finished: 3810 re

10 reviews found
Finished: 5370 reviews found
10 reviews found
Finished: 5380 reviews found
10 reviews found
Finished: 5390 reviews found
10 reviews found
Finished: 5400 reviews found
11 reviews found
Finished: 5410 reviews found
10 reviews found
Finished: 5420 reviews found
10 reviews found
Finished: 5430 reviews found
10 reviews found
Finished: 5440 reviews found
10 reviews found
Finished: 5450 reviews found
10 reviews found
Finished: 5460 reviews found
10 reviews found
Finished: 5470 reviews found
10 reviews found
Finished: 5480 reviews found
10 reviews found
Finished: 5490 reviews found
10 reviews found
Finished: 5500 reviews found
10 reviews found
Finished: 5510 reviews found
10 reviews found
Finished: 5520 reviews found
10 reviews found
Finished: 5530 reviews found
10 reviews found
Finished: 5540 reviews found
10 reviews found
Finished: 5550 reviews found
10 reviews found
Finished: 5560 reviews found
10 reviews found
Finished: 5570 reviews found
10 reviews found
Finished: 5580 re

HTTPError: 500 Server Error: Internal Server Error for url: https://api.proxycrawl.com/?token=8wJqpL2FErGiPGX7QWP6Aw&url=https%3A//www.amazon.com/TCL-50S425-inch-Smart-Roku/product-reviews/B07JK98NNQ%3FreviewerType%3Dall_reviews%26pageNumber%3D243

In [37]:
len(reviews)

7080

In [ ]:

print("Start to generate report")
# rev={'dates':dates, 'titles':titles, 'ratings':ratings, 'reviews':reviews, 'url':urls} #converting the reviews list into a dictionary

rev={'search_query':search_query_list, 'reviews' :reviews} #converting the reviews list into a dictionary
review_data=pd.DataFrame.from_dict(rev) #converting this dictionary into a dataframe

df = review_data.replace('\n','', regex=True)

writer = pd.ExcelWriter(SEARCH_QUERY+'_review.xlsx')
df.to_excel(writer, 'Sheet1', index=False)
writer.save()
print("Success")
